# CP Decomposition Robustness Analysis

This notebook demonstrates how to use the CP decomposition functionality from `decompose_cp.py`.


In [4]:
import sys
import os

# Adjust the path as needed to reach your project root from the notebook's location
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [5]:
from config import PROJECT_ROOT
from pathlib import Path

In [6]:
# Import necessary libraries
import tensorly as tl
import logging
from datetime import datetime
import json
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations, chain
from scipy.optimize import linear_sum_assignment
from typing import Dict, List, Tuple, Optional
from tensorly.cp_tensor import cp_to_tensor
import torch

# # Add the code directory to the path to import decompose_cp
# sys.path.append(str(Path.cwd() / 'code'))

# Import functions from decompose_cp
from decompose_cp import (
    load_sparse_tensor,
    run_cp_decomposition,
    compute_c1_unique_zones,
    compute_c2_unique_od_pairs,
    get_device,
    parse_rank_range
)

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Configure TensorLy to use PyTorch backend
tl.set_backend('pytorch')

print("Libraries imported successfully!")

2025-07-21 18:23:37,520 - INFO - Running on local environment
2025-07-21 18:23:37,521 - INFO - Using random state: 42
2025-07-21 18:23:37,596 - INFO - Using mps device for computation
2025-07-21 18:23:37,596 - INFO - Using Apple MPS (Metal) for acceleration - memory stats not available


Running in local environment - packages should be installed in local environment
Libraries imported successfully!


In [7]:
tensor_paths = [
    (5, str(PROJECT_ROOT) + "/data/processed/utrecht/odt_no_same_od_no_rare_od_fixed_thresh_normalizedPeaks/timehours/odt_processed_utrecht_hourly_weekday.npz"),
    (5, str(PROJECT_ROOT) + "/data/processed/utrecht/odt_no_same_od_no_rare_od_fixed_thresh_normalizedPeaks/timehours/odt_processed_utrecht_hourly_weekend.npz"),
    (6, str(PROJECT_ROOT) + "/data/processed/rotterdam/odt_no_same_od_no_rare_od_fixed_thresh_normalizedPeaks/timehours/odt_processed_rotterdam_hourly_weekday.npz"),
    (6, str(PROJECT_ROOT) + "/data/processed/rotterdam/odt_no_same_od_no_rare_od_fixed_thresh_normalizedPeaks/timehours/odt_processed_rotterdam_hourly_weekend.npz"),
]

results_path = str(PROJECT_ROOT) + "/data/results/decompositions/robustness/CP"

analysis_dir = None


# Get device information
device, device_name = get_device()
print(f"Using device: {device_name}")
print(f"Results will be saved to: {results_path}")

Using device: mps
Results will be saved to: /Users/peterfalterbaum/Documents/Nova/thesis local/implementation/public_implementation/data/results/decompositions/robustness/CP


In [8]:
def run_cp_robustness_analysis(tensor_path: str, ranks: list[int],
                               num_runs: int = 5, optimizer: str = 'ALS',
                               max_iter: int = 1000, tol: float = 1e-8,
                               init_methods_param: str = 'random', l1_reg: float = 0.0,
                               l2_reg: float = 0.0, top_n: int = 2,
                               use_percentage: bool = True, percentage: float = 0.2) -> None:
    """
    Run CP decomposition robustness analysis with multiple random seeds.
    """
    # Create results directory
    results_dir = Path(results_path)
    results_dir.mkdir(parents=True, exist_ok=True)

    # Create timestamped directory for this analysis
    global analysis_dir
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    identifier_tensor = "_".join(Path(tensor_path).stem.split("_")[-3:])
    analysis_dir = results_dir / f"{identifier_tensor}/{timestamp}"
    analysis_dir.mkdir(parents=True, exist_ok=True)

    logger.info(f"Starting robustness analysis with {num_runs} runs")
    logger.info(f"Results will be stored in: {analysis_dir}")

    # Load the tensor
    logger.info(f"Loading tensor from: {tensor_path}")
    tensor, origins, destinations, time_indices = load_sparse_tensor(
        tensor_path)

    # Extract tensor name from the path
    tensor_name = Path(tensor_path).stem

    # Run multiple decompositions with different random seeds
    for run_num in range(num_runs):
        logger.info(f"\n=== Starting Run {run_num + 1}/{num_runs} ===")

        # Generate a different random seed for each run
        random_seed = np.random.randint(0, 1000000)
        logger.info(f"Using random seed: {random_seed}")

        # Create run-specific directory
        run_dir = analysis_dir / f"run_{run_num + 1:03d}_seed_{random_seed}"

        # Use SVD initialization for the first run as baseline
        if run_num == 0:
            init_methods = 'svd'
            logger.info(f"Using SVD initialization for the first run")
            run_dir = analysis_dir / \
                f"svd_{run_num + 1:03d}_seed_{random_seed}"
        else:
            init_methods = init_methods_param

        run_dir.mkdir(exist_ok=True)

        try:
            # Run CP decomposition
            results_dict, run_results_dir = run_cp_decomposition(
                input_tensor=tensor,
                ranks=ranks,
                tensor_name=tensor_name,
                output_path=run_dir,
                optimizer=optimizer,
                max_iter=max_iter,
                tol=tol,
                init_methods=init_methods,
                l1_reg=l1_reg,
                l2_reg=l2_reg,
                top_n=top_n,
                use_percentage=use_percentage,
                percentage=percentage,
                origins=origins,
                destinations=destinations,
                random_state=random_seed
            )

            # Save run-specific information
            run_info = {
                "run_number": run_num + 1,
                "random_seed": random_seed,
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "tensor_path": tensor_path,
                "ranks": ranks,
                "optimizer": optimizer,
                "status": "completed"
            }

            with open(run_dir / "run_info.json", 'w') as f:
                json.dump(run_info, f, indent=4)

            logger.info(f"Run {run_num + 1} completed successfully")

        except Exception as e:
            logger.error(f"Error in run {run_num + 1}: {str(e)}")

            # Save error information
            error_info = {
                "run_number": run_num + 1,
                "random_seed": random_seed,
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "error": str(e),
                "status": "failed"
            }

            with open(run_dir / "error_info.json", 'w') as f:
                json.dump(error_info, f, indent=4)

    logger.info(
        f"\nRobustness analysis completed. Results stored in: {analysis_dir}")

    # Create summary file
    summary = {
        "analysis_timestamp": timestamp,
        "tensor_path": tensor_path,
        "tensor_name": tensor_name,
        "num_runs": num_runs,
        "ranks": ranks,
        "optimizer": optimizer,
        "analysis_dir": str(analysis_dir),
        "status": "completed"
    }

    with open(analysis_dir / "analysis_summary.json", 'w') as f:
        json.dump(summary, f, indent=4)

    return analysis_dir

In [9]:
def run_robustness_analysis(tensor_path, num_runs, ranks):
    """Run robustness analysis on the first tensor with example parameters."""

    analysis_dir = run_cp_robustness_analysis(
        tensor_path=tensor_path,
        ranks=ranks,
        num_runs=num_runs,  # Start with 3 runs for testing
        optimizer='ALS',
        max_iter=1000,
        tol=1e-8,
        init_methods_param='random',
        l1_reg=0.0,
        l2_reg=0.0
    )

    print(f"Analysis completed. Results in: {analysis_dir}")
    return analysis_dir

In [10]:
from tlviz.factor_tools import (
    normalise_cp_tensor,
    factor_match_score,
    degeneracy_score
)
from tensorly.cp_tensor import cp_to_tensor
import tensorly as tl
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import chain
from pathlib import Path
import numpy as np
import json
import sys
# Increase the recursion depth limit to handle deep import chains in libraries.
sys.setrecursionlimit(10_000)


def load_cp_runs(analysis_dir: Path, rank: int):
    """
    Load CP decomposition runs from a robustness directory.
    Returns a list of tuples: (run_name, error, cp_tensor).
    """
    runs = []
    for run_dir in chain(analysis_dir.glob("run_*"), analysis_dir.glob("svd_*")):
        for rank_dir in run_dir.rglob(f"rank_{rank}"):
            metrics_file = next(rank_dir.glob("*_metrics.json"), None)
            factors_file = next(rank_dir.glob("*_factors.npz"), None)
            if metrics_file and factors_file:
                with open(metrics_file, "r") as f:
                    metrics = json.load(f)
                data = np.load(factors_file, allow_pickle=True)
                weights = data.get("weights", data[data.files[0]])
                factors = data.get("factors", data[data.files[1]])
                error = metrics.get("normalized_frobenius_error", np.nan)
                runs.append((run_dir.name, error, (weights, factors)))
    return runs


def compare_cp_stability_tlviz(analysis_dir: Path, rank: int, num_best: int = 5):
    """
    Compute CP stability metrics using TLViz tools.

    This function calculates and visualizes:
      1) Pairwise Factor-Match Score
      2) Pairwise Tensor-Cosine Similarity
      3) Degeneracy score for each model

    Returns a dict with DataFrames for the two pairwise metrics and a dict for degeneracy.
    """
    # 1) Load all runs and select the best ones for comparison
    all_runs = load_cp_runs(analysis_dir, rank)
    if not all_runs:
        print(f"No runs found for rank {rank}")
        return {}

    # Sort by error to find the best models
    best_runs = sorted(all_runs, key=lambda x: x[1])[:num_best]

    # Ensure the SVD run is included for baseline comparison if it's not already in the best runs
    svd_run = next((r for r in all_runs if r[0].startswith("svd")), None)
    if svd_run and svd_run[0] not in [r[0] for r in best_runs]:
        best_runs.append(svd_run)

    # 3) Normalize each CP tensor and store it in a dictionary
    models = {name: normalise_cp_tensor(cp) for name, _, cp in best_runs}
    model_names = list(models.keys())

    # 4a) Compute pairwise Factor Match Score
    fms_matrix = np.zeros((len(model_names), len(model_names)))
    for i, name1 in enumerate(model_names):
        for j, name2 in enumerate(model_names):
            if i >= j:
                score = factor_match_score(models[name1], models[name2])
                fms_matrix[i, j] = score
                fms_matrix[j, i] = score
    df_fms = pd.DataFrame(fms_matrix, index=model_names, columns=model_names)

    # 4b) Compute pairwise Tensor Cosine Similarity
    reconstructed_models = {name: cp_to_tensor(
        cp) for name, cp in models.items()}
    cos_matrix = np.zeros((len(model_names), len(model_names)))
    for i, name1 in enumerate(model_names):
        for j, name2 in enumerate(model_names):
            if i >= j:
                t1 = reconstructed_models[name1]
                t2 = reconstructed_models[name2]
                score = tl.sum(t1 * t2) / (tl.norm(t1) * tl.norm(t2))
                cos_matrix[i, j] = score
                cos_matrix[j, i] = score
    df_cos = pd.DataFrame(cos_matrix, index=model_names, columns=model_names)

    # 4c) Compute Degeneracy for each model
    deg = {name: float(degeneracy_score(cp)) for name, cp in models.items()}

    # 5) Plot the results
    fig, axes = plt.subplots(1, 2, figsize=(22, 9))
    fig.suptitle(f"CP Stability Analysis for Rank {rank}", fontsize=16)

    sns.heatmap(df_fms, annot=True, fmt=".3f", cmap="viridis", ax=axes[0])
    axes[0].set_title("Factor-Match Score")
    axes[0].tick_params(axis='x', rotation=45, ha='right')
    axes[0].tick_params(axis='y', rotation=0)

    sns.heatmap(df_cos, annot=True, fmt=".3f", cmap="viridis", ax=axes[1])
    axes[1].set_title("Tensor Cosine Similarity")
    axes[1].tick_params(axis='x', rotation=45, ha='right')
    axes[1].tick_params(axis='y', rotation=0)

    plt.tight_layout(rect=[0, 0, 1, 0.96])

    # Save the figure
    out_png = analysis_dir / f"stability_summary_rank_{rank}.png"
    plt.savefig(out_png, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print(f"\nSaved stability plot to {out_png}")

    # Print degeneracy scores
    print(f"Degeneracy scores for Rank {rank}:")
    for name, score in sorted(deg.items()):
        print(f"  - {name}: {score:.4f}")

    return {
        "factor_match": df_fms.to_dict(),
        "tensor_cosine": df_cos.to_dict(),
        "degeneracy": deg,
    }

In [ ]:
collected_stability_results = []

for rank, tensor_path in tensor_paths:
    # 1) kick off the robustness job *and* capture its output directory
    result_dir = run_robustness_analysis(tensor_path, 5, [rank])
    #    ^-- this should be a Path pointing to "…/20250626_221737" etc.

    # 2) now pass *that* directory into your comparer
    stability_results = compare_cp_stability_tlviz(
        result_dir, rank=rank, num_best=5)
    collected_stability_results.append(stability_results)

# 3) finally write out your collected stats
with open('stability_results.json', 'w') as f:
    json.dump(collected_stability_results, f)

## individual decomposition test


In [ ]:
# _, tensor_path = tensor_paths[3]
# tensor_path

'/Users/peterfalterbaum/Documents/Nova/thesis local/implementation/rocket riding/data/processed/rotterdam/odt_no_same_od_no_rare_od_fixed_thresh_normalizedPeaks/timehours/odt_processed_rotterdam_hourly_weekend.npz'

In [ ]:
# rank = 4
# run_robustness_analysis(tensor_path, 30, [rank])
# # Example usage:
# # analysis_dir = Path('/Users/peterfalterbaum/Documents/Nova/thesis local/implementation/rocket riding/data/results/decompositions/robustness/CP/robustness_analysis_20250625_132752')
# stability_results = compare_cp_stability(
#     Path(analysis_dir), rank=rank, num_best=7)  # , tensor_path=Path(tensor_path)
# collected_stability_results.append(stability_results)

In [ ]:
# stability_results = compare_cp_stability(Path(analysis_dir), rank=rank, num_best=7)  # , tensor_path=Path(tensor_path)